# Imports

In [1]:
# standard import
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
# visualization
import matplotlib.pyplot as plt
import seaborn as sns
# stats
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
# aquire
from env import user, password, host
import os
# notebook formatting
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")
# plotting defaults
plt.rc('figure', figsize=(13, 7))
plt.style.use('seaborn-whitegrid')
plt.rc('font', size=16)

### Using your store items data:

In [2]:
def get_db_url(database):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'

def get_store_data():
    '''
    Returns a dataframe of all store data in the 
    tsa_item_demand database and saves a local copy 
    as a csv file.
    '''
    query = '''
    SELECT *
    FROM items
    JOIN sales USING(item_id)
    JOIN stores USING(store_id) 
    '''
    df = pd.read_sql(query, get_db_url('tsa_item_demand'))
    df.to_csv('tsa_item_demand.csv', index=False)
    return df

def wrangle_store_data():
    filename = 'tsa_store_data.csv'
    if os.path.isfile(filename):
        df = pd.read_csv(filename, index_col=0)
    else:
        df = get_store_data()
    return df

In [3]:
df = get_store_data()
df.head()

,store_id,item_id,item_upc14,item_upc12,item_brand,item_name,item_price,sale_id,sale_date,sale_amount,store_address,store_zipcode,store_city,store_state
0,1,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,1,2013-01-01,13,12125 Alamo Ranch Pkwy,78253,San Antonio,TX
1,1,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,2,2013-01-02,11,12125 Alamo Ranch Pkwy,78253,San Antonio,TX
2,1,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,3,2013-01-03,14,12125 Alamo Ranch Pkwy,78253,San Antonio,TX
3,1,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,4,2013-01-04,13,12125 Alamo Ranch Pkwy,78253,San Antonio,TX
4,1,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,5,2013-01-05,10,12125 Alamo Ranch Pkwy,78253,San Antonio,TX


In [4]:
# shape
df.shape

(913000, 14)

In [5]:
# info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   store_id       913000 non-null  int64  
 1   item_id        913000 non-null  int64  
 2   item_upc14     913000 non-null  object 
 3   item_upc12     913000 non-null  object 
 4   item_brand     913000 non-null  object 
 5   item_name      913000 non-null  object 
 6   item_price     913000 non-null  float64
 7   sale_id        913000 non-null  int64  
 8   sale_date      913000 non-null  object 
 9   sale_amount    913000 non-null  int64  
 10  store_address  913000 non-null  object 
 11  store_zipcode  913000 non-null  object 
 12  store_city     913000 non-null  object 
 13  store_state    913000 non-null  object 
dtypes: float64(1), int64(4), object(9)
memory usage: 97.5+ MB


In [6]:
# df tail
df.tail()

,store_id,item_id,item_upc14,item_upc12,item_brand,item_name,item_price,sale_id,sale_date,sale_amount,store_address,store_zipcode,store_city,store_state
912995,10,50,47445919221,47445919221,Choice,Choice Organic Teas Black Tea Classic Black - ...,5.2,912996,2017-12-27,63,8503 NW Military Hwy,78231,San Antonio,TX
912996,10,50,47445919221,47445919221,Choice,Choice Organic Teas Black Tea Classic Black - ...,5.2,912997,2017-12-28,59,8503 NW Military Hwy,78231,San Antonio,TX
912997,10,50,47445919221,47445919221,Choice,Choice Organic Teas Black Tea Classic Black - ...,5.2,912998,2017-12-29,74,8503 NW Military Hwy,78231,San Antonio,TX
912998,10,50,47445919221,47445919221,Choice,Choice Organic Teas Black Tea Classic Black - ...,5.2,912999,2017-12-30,62,8503 NW Military Hwy,78231,San Antonio,TX
912999,10,50,47445919221,47445919221,Choice,Choice Organic Teas Black Tea Classic Black - ...,5.2,913000,2017-12-31,82,8503 NW Military Hwy,78231,San Antonio,TX


In [7]:
# nulls
(df.isnull().sum() != 0).sum()

0

In [8]:
# how many unique stores?
df.store_id.nunique()

10

In [9]:
# how many unique items?
df.item_id.nunique()

50

In [10]:
# how many unique sales dates
df.sale_date.nunique()

1826

In [11]:
# number of unique timestamps * number of unique items * number of unique stores
(1826 * 10 * 50) == df.shape[0]

True

In [30]:
# what format is date and time?
print(df.sale_date.min())
print(df.sale_date.max())

In [25]:
# convert date and time
df.sale_date = pd.to_datetime(df.sale_date)

In [26]:
# verify date and time format change
print(df.sale_date.min())
print(df.sale_date.max())

Timestamp('2013-01-01 00:00:00')

In [28]:
# reassign this new formatted datetime to our index
df = df.set_index('sale_date').sort_index()
df.head()

,store_id,item_id,item_upc14,item_upc12,item_brand,item_name,item_price,sale_id,sale_amount,store_address,store_zipcode,store_city,store_state
sale_date,,,,,,,,,,,,,
2013-01-01,1,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,1,13,12125 Alamo Ranch Pkwy,78253,San Antonio,TX
2013-01-01,3,17,88313590791,88313590791,Ducal,Ducal Refried Red Beans,1.16,295813,26,2118 Fredericksburg Rdj,78201,San Antonio,TX
2013-01-01,10,7,70177154004,70177154004,Twinings Of London,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,125995,32,8503 NW Military Hwy,78231,San Antonio,TX
2013-01-01,3,18,21200725340,21200725340,Scotch,Scotch Removable Clear Mounting Squares - 35 Ct,4.39,314073,45,2118 Fredericksburg Rdj,78201,San Antonio,TX
2013-01-01,3,19,41520035646,41520035646,Careone,Careone Family Comb Set - 8 Ct,0.74,332333,34,2118 Fredericksburg Rdj,78201,San Antonio,TX


In [4]:
df.index.min(), df.index.max()

(0, 912999)

In [8]:
train = df[:2015]
validate = df['2016']
test = df['2017']

KeyError: '2016'

In [ ]:
train.head()

In [ ]:
train.resample('D').sales_total.mean().plot()
validate.resample('D').sales_total.mean().plot()
test.resample('D').sales_total.mean().plot()

### 1. Convert date column to datetime format.

### 2. Plot the distribution of sale_amount and item_price.

### 3. Set the index to be the datetime variable.

### 4. Add a 'month' and 'day of week' column to your dataframe.

### 5. Add a column to your dataframe, sales_total, which is a derived from sale_amount (total items) and item_price.

### 6. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions and be able to re-run the functions and get the same results.